In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [2]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [3]:
import time
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, DataCollatorWithPadding, AutoModel, TrainingArguments, Trainer
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchmetrics.classification import MultilabelAccuracy
from datasets import Dataset
import pickle
import random
import evaluate


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('/kaggle/input/dataset/train.csv')
test_df = pd.read_csv('/kaggle/input/dataset/test.csv')
train_df.head()

,id,comment,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
0,0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,0,0,0,0,0,0,0,0,0,0,0,0
1,1,บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแ...,0,0,0,0,0,0,0,0,0,0,0,0
2,2,เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซน...,0,0,0,0,0,0,0,0,0,0,0,0
3,3,ปัญหาน้ำท่วมในซอยสวนพลู 1,0,0,0,0,0,0,0,0,0,0,0,0
4,4,1. ซ่อมสายไฟ กรีดขวางทางเท้า,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
train_df.describe()

,id,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
count,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000
mean,102310.500000,0.209635,0.019969,0.000049,0.016034,0.000117,0.167944,0.034542,0.000406,0.024176,0.018478,0.000059,0.023839
std,59069.427728,0.407049,0.139892,0.006991,0.125608,0.010829,0.373817,0.182616,0.020136,0.153597,0.134672,0.007658,0.152548
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51155.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,102310.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,153465.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,204621.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
def drop_rows(df):
    # drop rows with missing values, duplicates
    print(f"Before dropna: {df.shape}")
    df.dropna(inplace=True)
    df.drop_duplicates("comment", keep="first", inplace=True)
    df = df.reset_index(drop=True)
    print(f"After dropna: {df.shape}")

drop_rows(train_df)
drop_rows(test_df)
print(train_df.shape, test_df.shape)

Before dropna: (204622, 14)
After dropna: (204622, 14)
Before dropna: (10810, 2)
After dropna: (10810, 2)
(204622, 14) (10810, 2)


In [8]:
contacts = train_df.columns[2:]

train_df['label'] = train_df[contacts].apply(lambda x: list(x), axis=1)
train_df.drop(contacts, axis=1, inplace=True)
train_df.drop("id", axis=1, inplace=True)

test_df.drop("id", axis=1, inplace=True)
train_df.head()

,comment,label
0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซน...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,ปัญหาน้ำท่วมในซอยสวนพลู 1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,1. ซ่อมสายไฟ กรีดขวางทางเท้า,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [9]:
train_df['label'].value_counts()

label
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    102333
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]     42212
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]     32543
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]      5803
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]      4598
                                         ...  
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]         1
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]         1
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]         1
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]         1
[0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]         1
Name: count, Length: 73, dtype: int64

In [10]:
train_df = train_df.loc[train_df['label'].apply(lambda x: sum(x) != 0)]
print(train_df.shape)
train_df['label'].value_counts()

(102289, 2)


label
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    42212
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]    32543
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]     5803
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]     4598
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]     3993
                                        ...  
[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1]        1
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]        1
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]        1
[0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]        1
[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]        1
Name: count, Length: 72, dtype: int64

In [11]:
contacts

Index(['สำนักงานตำรวจแห่งชาติ', 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
       'สภาเด็กและเยาวชนกรุงเทพมหานคร', 'กรมควบคุมมลพิษ', 'กรมสรรพสามิต',
       'การไฟฟ้านครหลวง', 'กรมทางหลวง', 'สำนักงานประกันสุขภาพแห่งชาติ',
       'การประปานครหลวง', 'คณะกรรมการการพัฒนาเศรษฐกิจ',
       'กระทรวงการท่องเที่ยวและกีฬา', 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200'],
      dtype='object')

In [12]:
train_data = train_df.copy().to_numpy()
test_data = test_df.copy().to_numpy()

label_2_idx = dict(zip(contacts, range(len(contacts))))
idx_2_label = dict(zip(range(len(contacts)), contacts))

print("Create Mappings")
display(idx_2_label)
display(label_2_idx)

# print("Before Mappings")
# display(data[:, 1])
# data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'สำนักงานตำรวจแห่งชาติ',
 1: 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
 2: 'สภาเด็กและเยาวชนกรุงเทพมหานคร',
 3: 'กรมควบคุมมลพิษ',
 4: 'กรมสรรพสามิต',
 5: 'การไฟฟ้านครหลวง',
 6: 'กรมทางหลวง',
 7: 'สำนักงานประกันสุขภาพแห่งชาติ',
 8: 'การประปานครหลวง',
 9: 'คณะกรรมการการพัฒนาเศรษฐกิจ',
 10: 'กระทรวงการท่องเที่ยวและกีฬา',
 11: 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200'}

{'สำนักงานตำรวจแห่งชาติ': 0,
 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย': 1,
 'สภาเด็กและเยาวชนกรุงเทพมหานคร': 2,
 'กรมควบคุมมลพิษ': 3,
 'กรมสรรพสามิต': 4,
 'การไฟฟ้านครหลวง': 5,
 'กรมทางหลวง': 6,
 'สำนักงานประกันสุขภาพแห่งชาติ': 7,
 'การประปานครหลวง': 8,
 'คณะกรรมการการพัฒนาเศรษฐกิจ': 9,
 'กระทรวงการท่องเที่ยวและกีฬา': 10,
 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200': 11}

In [13]:
def set_random_seed(seed: int):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(42)

In [14]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, revision='main', model_max_length=128)

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [15]:
X_train_tokens = tokenizer(train_data[:,0].tolist(), truncation=True, padding=True)
X_test_tokens = tokenizer(test_data[:,0].tolist(), truncation=True, padding=True)
num_labels = len(label_2_idx)

In [16]:
y_train = np.array(train_data[:,1].tolist(), dtype=np.float32)
y_train = torch.tensor(y_train, dtype = torch.float32)

y_test = np.array([[0] * 12 for _ in range(test_data.shape[0])], dtype=np.float32)
y_test = torch.tensor(y_test, dtype = torch.float32)

y_train.shape, y_test.shape

(torch.Size([102289, 12]), torch.Size([10810, 12]))

In [17]:
train_dataset = Dataset.from_dict({
    "input_ids": X_train_tokens["input_ids"],
    "attention_mask": X_train_tokens["attention_mask"],
    "labels": y_train
})

test_dataset = Dataset.from_dict({
    "input_ids": X_test_tokens["input_ids"],
    "attention_mask": X_test_tokens["attention_mask"],
    "labels": y_test
})

In [18]:
num_labels = 12
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="multi_label_text_classification_wangchanberta_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

In [20]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-21-217194651456>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.066400,0.240364,0.927706,0.000000,0.000000,0.000000
2,0.055600,0.255033,0.925632,0.000000,0.000000,0.000000
3,0.049600,0.264103,0.925971,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
 

TrainOutput(global_step=9591, training_loss=0.06604290530574967, metrics={'train_runtime': 4840.0292, 'train_samples_per_second': 63.402, 'train_steps_per_second': 1.982, 'total_flos': 2.018683738824192e+16, 'train_loss': 0.06604290530574967, 'epoch': 3.0})

In [23]:
import shutil

shutil.make_archive("/kaggle/working/checkpoint-9591", "zip", "/kaggle/working/multi_label_text_classification_wangchanberta_model/checkpoint-9591")


'/kaggle/working/checkpoint-9591.zip'

In [24]:
predictions = trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
logits = torch.tensor(predictions.predictions)
probs = torch.sigmoid(logits)
pred_labels = (probs > 0.5).int().tolist()
pred_df = pd.DataFrame(pred_labels, columns=contacts)

test_df = pd.read_csv('/kaggle/input/dataset/test.csv')
output_df = pd.concat([test_df["id"], pred_df], axis=1)

In [26]:
output_df.to_csv('submission.csv', index=False)

In [27]:
output_df

,id,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10805,10805,0,0,0,0,0,0,0,0,0,1,0,0
10806,10806,1,0,0,0,0,0,0,0,0,0,0,0
10807,10807,0,0,0,0,0,0,0,0,0,1,0,0
10808,10808,1,0,0,0,0,0,0,0,0,0,0,0
